In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import weather_plots as wp  # Assuming this contains your plotting functions
import matplotlib.patches as patches
from fetch_data import fetch_weather_data
from process_data import process_weather_data

def determine_weather(row, sunny_threshold, rainy_min_temp_threshold, rainy_humidity_threshold):
    if row['Max Temperature (C)'] > sunny_threshold:
        return 'Sunny'
    elif row['Min Temperature (C)'] < rainy_min_temp_threshold and row['Max Humidity (%)'] > rainy_humidity_threshold:
        return 'Rainy'
    else:
        return 'Partly Cloudy'

# Dropdown for city selection
city_dropdown = widgets.Dropdown(
    options=['Sydney', 'New York', 'London', 'Beijing'],
    value='Sydney',
    description='City:'
)

# Button to fetch and update weather data
update_button = widgets.Button(
    description='Update Weather',
    button_style='success'
)

output = widgets.Output()

# Function to update weather data and plots
def update_weather(b):
    city = city_dropdown.value
    api_key = 'd0de46031ad7410d0c72c6063690e1d0'  # Replace with your OpenWeatherMap API key
    current_data, forecast_data = fetch_weather_data(api_key, city)
    current_df, hourly_df, daily_df = process_weather_data(current_data, forecast_data)
    daily_df['Weather'] = daily_df.apply(determine_weather, axis=1, args=(25, 15, 80))

    with output:
        output.clear_output()
        fig, axs = plt.subplots(1, 3, figsize=(15, 4))
        fig.tight_layout(pad=5.0)

        # Generate plots
        wp.create_forecast_card(axs[0], daily_df.iloc[0]['Date'], daily_df.iloc[0]['Max Temperature (C)'], daily_df.iloc[0]['Min Temperature (C)'], daily_df.iloc[0]['Weather'])
        wp.plot_scatter(hourly_df, 'Temperature (C)', 'Wind Speed (m/s)', axs[1], title="Wind Speed vs Temperature")
        wp.plot_min_max(daily_df, 'Date', 'Min Temperature (C)', 'Max Temperature (C)', axs[2], title="Daily Min/Max Temperature")

        plt.show()

update_button.on_click(update_weather)
display(city_dropdown, update_button, output)
update_weather(None)
